# Scrap to save in csv and mysql

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import mysql.connector

# Connect to the MySQL database
try:
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="books_db"
    )
    print("Connection to the database was successful!")
except mysql.connector.Error as e:
    print(f"Error connecting to MySQL: {e}")
    exit()

# Create a cursor object to interact with the database
cursor = db.cursor()

# Create a table to store the book information
cursor.execute("""
    CREATE TABLE IF NOT EXISTS books (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        rating VARCHAR(10),
        price VARCHAR(20),
        availability VARCHAR(50),
        category VARCHAR(255),
        image_url VARCHAR(255)
    )
""")

# Create a CSV file to store the book information
with open("data/books_info.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Rating", "Price", "Availability", "Category", "Image URL"])

    base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
    
    for page_num in range(1, 51):
        # Request the page
        res = requests.get(base_url.format(page_num))

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find all products on the page
        products = soup.find_all('article', class_='product_pod')

        # Loop through each product and extract the information
        for product in products:
            # Extract the title
            title = product.find('h3').find('a').get('title')

            # Extract the rating
            rating = product.find('p', class_='star-rating')['class'][1]

            # Extract the price
            price = product.select('.price_color')[0].get_text()

            # Extract the availability
            availability = product.select('.availability')[0].get_text().strip()

            # Extract the URL slug
            url_slug = product.find('a')['href']

            # Extract the image URL
            image_url = f"http://books.toscrape.com/{product.find('img')['src'][5:]}"

            # Extract the category from the URL slug
            category = url_slug.split('/')[2] if len(url_slug.split('/')) >= 3 else ''

            # Write the information to the CSV file
            writer.writerow([title, rating, price, availability, category, image_url])

            # Insert the information into the database
            cursor.execute("INSERT INTO books (title, rating, price, availability, category, image_url) VALUES (%s, %s, %s, %s, %s, %s)", (title, rating, price, availability, category, image_url))
            db.commit()

print("Data extraction and insertion complete!")

# Close the connection to the database
db.close()


Connection to the database was successful!
Data extraction and insertion complete!


# To display only a certain number of rows from table in database

In [2]:
# Connect to the MySQL database
try:
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="books_db"
    )
    print("Connection to the database was successful!")
except mysql.connector.Error as e:
    print(f"Error connecting to MySQL: {e}")
    exit()
    
# Create a cursor object to interact with the database
cursor = db.cursor()

# Select all the rows in the books table
cursor.execute("SELECT * FROM books")
result = cursor.fetchall()

# Display the first 10 rows
for row in result[:10]:
    print(row)

# Close the connection to the database
db.close()

Connection to the database was successful!
(1, bytearray(b'A Light in the Attic'), bytearray(b'Three'), bytearray(b'\xc3\x82\xc2\xa351.77'), bytearray(b'In stock'), bytearray(b''), bytearray(b'http://books.toscrape.com/dia/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'))
(2, bytearray(b'Tipping the Velvet'), bytearray(b'One'), bytearray(b'\xc3\x82\xc2\xa353.74'), bytearray(b'In stock'), bytearray(b''), bytearray(b'http://books.toscrape.com/dia/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg'))
(3, bytearray(b'Soumission'), bytearray(b'One'), bytearray(b'\xc3\x82\xc2\xa350.10'), bytearray(b'In stock'), bytearray(b''), bytearray(b'http://books.toscrape.com/dia/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg'))
(4, bytearray(b'Sharp Objects'), bytearray(b'Four'), bytearray(b'\xc3\x82\xc2\xa347.82'), bytearray(b'In stock'), bytearray(b''), bytearray(b'http://books.toscrape.com/dia/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg'))
(5, bytearray(b'Sapiens: A Brief History of Humankind